In [1]:
https://colab.research.google.com/drive/1ChMuIV_X3A2NGf3yhSMDOTQtsCbX74z5?usp=sharing

SyntaxError: invalid decimal literal (ipython-input-2915169418.py, line 1)

In [1]:
import pandas as pd

In [5]:
# --- 1. DATA PREPARATION ---

# Load the dataframes
fear_greed_df = pd.read_csv("fear_greed_index.csv")
historical_data_df = pd.read_csv("historical_data.csv")

# Get initial row count for data coverage calculation
original_rows = len(historical_data_df)

In [6]:
# Standardize Date Columns for Merging
# Correct date parsing for historical data using explicit format
historical_data_df['Date'] = pd.to_datetime(
    historical_data_df['Timestamp IST'],
    format="%d-%m-%Y %H:%M"
).dt.strftime('%Y-%m-%d')
fear_greed_df['Date'] = pd.to_datetime(fear_greed_df['date']).dt.strftime('%Y-%m-%d')

In [7]:
# --- 2. MERGING AND TOP 5 IDENTIFICATION ---

# Merge PnL data with Fear & Greed Classification (Left Merge)
merged_df = pd.merge(
    historical_data_df,
    fear_greed_df[['Date', 'classification']],
    on='Date',
    how='left'
)

# Calculate Total PnL for all coins
total_pnl_by_coin = merged_df.groupby('Coin')['Closed PnL'].sum().reset_index()
total_pnl_by_coin.rename(columns={'Closed PnL': 'Total PnL (USD)'}, inplace=True)

In [8]:
# Identify the Top 5 most profitable coins
top_5_by_pnl_sorted = total_pnl_by_coin.sort_values(
    by='Total PnL (USD)',
    ascending=False
)
top_5_coins = top_5_by_pnl_sorted.head(5)['Coin'].tolist()

In [9]:
# --- 3. DATA COVERAGE CALCULATION ---

# Filter the merged data for the identified Top 5 coins
filtered_coin_df = merged_df[merged_df['Coin'].isin(top_5_coins)].copy()
rows_after_coin_filter = len(filtered_coin_df)

# Calculate data coverage
coin_filter_pct = (rows_after_coin_filter / original_rows) * 100

print("--- Data Coverage Report ---")
print(f"Original Number of Trades (Rows): {original_rows}")
print(f"Top 5 Most Profitable Coins: {top_5_coins}")
print(f"Rows covered by Top 5 Coins: {rows_after_coin_filter}")
print(f"Percentage of Original Trades Covered: {coin_filter_pct:.2f}%\n")

--- Data Coverage Report ---
Original Number of Trades (Rows): 211224
Top 5 Most Profitable Coins: ['@107', 'HYPE', 'SOL', 'ETH', 'BTC']
Rows covered by Top 5 Coins: 145910
Percentage of Original Trades Covered: 69.08%



In [10]:
# --- 4. AVERAGE DAILY PNL CALCULATION ---

# Remove unclassified days (Equivalent to Inner Join on Classified Days)
final_filtered_df = filtered_coin_df.dropna(subset=['classification']).copy()

# Calculate the Total PnL for each Coin/Classification on a given Day
daily_pnl = final_filtered_df.groupby(['Date', 'Coin', 'classification'])['Closed PnL'].sum().reset_index()

# Calculate the Average Daily PnL across all days for each Coin/Classification
avg_daily_pnl = daily_pnl.groupby(['Coin', 'classification'])['Closed PnL'].mean().reset_index()
avg_daily_pnl.rename(columns={'Closed PnL': 'Avg Daily PnL (USD)'}, inplace=True)

In [11]:
# Pivot the table for final output structure
pnl_table = avg_daily_pnl.pivot(index='classification', columns='Coin', values='Avg Daily PnL (USD)')

In [12]:
# Reorder rows for presentation
row_order = ['Extreme Greed', 'Greed', 'Neutral', 'Fear', 'Extreme Fear']
pnl_table = pnl_table.reindex(index=row_order)

In [13]:
# --- 5. AGGREGATION CALCULATIONS ---

# 5.1. Mathematical Aggregation
math_agg = pnl_table.mean(axis=1).sort_values(ascending=False).round(2)

In [14]:
# 5.2. Semantic Aggregation
# Find the sentiment that yields the max PnL for each coin
max_pnl_sentiment = pnl_table.idxmax(axis=0)
# Count how many coins peak at each sentiment
semantic_agg = max_pnl_sentiment.value_counts().sort_values(ascending=False)

In [15]:
# --- 6. PRINT FINAL AGGREGATIONS ---

print("--- Final Average Daily PnL Table (Raw) ---")
print(pnl_table.to_string(float_format='${:,.2f}'.format))
print("\n" + "="*50)

print("--- Mathematical Aggregation (Largest PnL Magnitude) ---")
print("Average PnL across all 5 assets for each sentiment class.")
print(math_agg.to_string(float_format='${:,.2f}'.format))
print("\n" + "="*50)

print("--- Semantic Aggregation (Majority Peak Strategy) ---")
print("Count of Assets for which this Sentiment is the Highest PnL Day.")
print(semantic_agg.to_string())

--- Final Average Daily PnL Table (Raw) ---
Coin                  @107       BTC        ETH       HYPE        SOL
classification                                                       
Extreme Greed   $60,261.18 $1,483.57   $-538.67  $6,141.79  $1,019.35
Greed           $17,666.89 $1,121.12  $2,700.28  $4,056.28  $9,033.80
Neutral          $8,782.08 $3,531.50  $1,431.96 $11,542.32 $13,789.83
Fear              $-320.89 $8,618.74 $10,547.88 $19,542.01 $17,657.01
Extreme Fear   $-12,371.40   $254.74 $19,594.30 $43,825.80 $11,686.61

--- Mathematical Aggregation (Largest PnL Magnitude) ---
Average PnL across all 5 assets for each sentiment class.
classification
Extreme Greed   $13,673.44
Extreme Fear    $12,598.01
Fear            $11,208.95
Neutral          $7,815.53
Greed            $6,915.67

--- Semantic Aggregation (Majority Peak Strategy) ---
Count of Assets for which this Sentiment is the Highest PnL Day.
Fear             2
Extreme Fear     2
Extreme Greed    1
